# Google Drive Auto Folder Creator<br>

This application is done for educational facilities. The instructor can use it  to create A main folder on google drive for the subject with sub-folders where each sub-folder is unique for the student to upload his/her work on, and get marked by the instructor. As well as exporting an excel sheet that contains metadata to all created folder.   The user (instructor) is using a simple guided GUI in order to create the folders. 
<br>
#### Done By: Omar Marouf <br>
<li> <a href = "https://github.com/mrprince22/"> Github </a> </li>
<li> <a href ="https://www.linkedin.com/in/ooshiba/"> LinkedIn </a></li>
<br>

In [69]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet

In [70]:
# pip install pydrive --quiet

In [71]:
# pip install pyautogui --quiet

In [72]:
from __future__ import print_function
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import os
import datetime
import time
from tkinter import *
from tkinter.ttk import *
from tkinter import Tk, Label, Button, messagebox
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")
font = ("Cairo", 15)
bg = "#2A2A2A"

In [74]:
# Initializing Connection ('client_secrets' file MUST exists in the same path)
gauth = GoogleAuth()           
drive = GoogleDrive(gauth)

In [75]:
def read_file(el_path: str):
"""
Function to read file after selecting it
"""
    global data
    data = pd.read_excel(el_path)
    print(data.head())
    
    
def browse():
    """
    Function called when pressing the browse button
    """
    filename = filedialog.askopenfilename(title = "Select an Excel Sheet:",filetypes=[('Excel Files', '*.xlsx')])
    global el_path
    el_path = filename
    file_itself = filename.split("/")[-1]
    label_file_explorer.configure(text="Selected File:"+file_itself)
    read_file(el_path)
    
def msgbox(title="Success", message="Service Created Successfully"):
    """
    Show A message box after certain condition is met
    """
    messagebox.showinfo(title, message)
    
def get_input():
    """
    Create All files after Entering the root.
    """
    global main_root
    main_root = root_path.get("1.0","end-1c").split("/")[-1]
    create_all()

    
def export():
    """
    Export the output excel file (file with all details on created folders) to the same path as the input data is stored in
    """
    
    exp_path = el_path.split("/")
    folder_name = (exp_path.pop()).split(".")[0]
    exp_path = "/".join(exp_path)
    now = datetime.datetime.now()
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    date_time = pd.to_datetime(date_time)
    new_dir = exp_path + "/" + folder_name.replace(" ", "-")
    if not os.path.isdir(new_dir):
        os.mkdir(new_dir)
    full_path = f"{new_dir}/Output-{now}.xlsx"
    out.to_excel(full_path, index = False)
    msgbox(message = "Exprted Successfully")
    print("Exported")
    os.system(f"open {new_dir}") # change open to start if you're using windows
    
    
    
def Create_Service():
    """
    Initializing service between the authorized emails and googe drive api
    
    """
    client_secret_file = "credentials.json" # client_secrets file downloaded from google api
    api_name = "drive"
    api_version = 'v3'
    scopes = [['https://www.googleapis.com/auth/drive']]
    
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            try:
                cred = pickle.load(token)
            except EOFError:
                    cred = list()

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        global service
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        msgbox()
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None
    

In [76]:
def create_folder(root : str ,name : str, email : str, id_: int)-> dict :
    
    """
     Parameters :
     'root' : {str : id of the root folder (folder to create your other folders)}
     'name' : {str : name of the folder will be created}
     'email': {str: email address to give write access to}
     'id_'  : {int: id of the student}

    """
    body = {
        'name' : name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents' : [root]
    }

    folder = service.files().create(body = body).execute()
    folder_id = folder['id']

    payload = { # defines the share access details of the created folder (editor by default)
            "role": "writer",
            "type": "user",
            "emailAddress" : f"{email}"
        }
    service.permissions().create(fileId = folder_id, body = payload).execute()
    msg = f"Created folder {name} with write access to {email}"
    print(msg)

    now = datetime.datetime.now()
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    date_time = pd.to_datetime(date_time)
    ret = { # Fileds of the output excel sheet
        'folder_name' : name,
        'folder_email' : email,
        'folder_id' : folder_id,
        'time_created' : date_time,
        "Student_ID" : id_
    }

    return ret

In [79]:
def create_all():
    progress['value'] = 0
    window.update_idletasks()
    root = main_root
    global out
    out = pd.DataFrame(columns = ['Timestamp', 'Folder_Name', "Student_ID", 'Folder_Link', 'Email_Address'])
    name_mail_id = [[row[1][0], row[1][1], row[1][2]] for row in data.iterrows()]
    length = len(name_mail_id)
    for name,email,id_ in name_mail_id:
        file = create_folder(root,name,email,id_)
        out = out.append({'Timestamp':file['time_created'],"Student_ID" : file["Student_ID"], 'Folder_Name':file['folder_name'], 'Folder_Link' : f"https://drive.google.com/drive/folders/{file['folder_id']}", 'Email_Address' : file['folder_email']}, ignore_index=True)
        progress['value'] += 100 / length
        window.update_idletasks()
    progress['value'] = 100
    window.update_idletasks()
    msgbox(message="Created Folders Successfully")


In [80]:
window = Tk()

window.title('Auto Drive Folder Ceator')

window.geometry("450x650")
# window.resizable(False, False)
window.config(background = bg)

label_file_explorer = Label(window,text = "Selected File:",
                            width = 100, height = 10,fg = "white", bg = bg, font = ("Cairo", 25))

button_explore = Button(window,text = "Browse Files",command = browse, highlightbackground=bg, font = font, height = 2, width = 12)
button_create_service = Button(window,text = "Link Your Account",command = Create_Service, highlightbackground = bg, font = font, height =2)

label_file_explorer.grid(column = 5, row = 2)
label_file_explorer.place(relx = 0.5, rely = 0.1, anchor = 'center')

button_create_service.grid(column = 1,row = 4)
button_create_service.place(relx = 0.5, rely = 0.28, anchor = 'center')

button_explore.grid(column = 1, row = 4)
button_explore.place(relx = 0.5, rely = 0.42, anchor = 'center')

enter_root = Label(text = "Copy & Paste The Main Drive Folder:", bg = bg, fg = 'white', font = ('Cairo', 20))
enter_root.grid()
enter_root.place(relx = 0.5, rely = 0.6, anchor = CENTER)

root_path = Text(window, width = 42, height = 2)
root_path.grid(column = 2, row = 5)
root_path.place(relx = 0.5, rely = 0.7, anchor = CENTER)

get_root = Button(window, text = "Start", font = font, highlightbackground = bg, command = get_input, width = 4, height = 1)
get_root.grid()
get_root.place(relx = 0.5, rely = 0.78, anchor= CENTER)

s = Style(window)
s.theme_use('clam')
s.configure("green.Horizontal.TProgressbar", foreground = 'green', background='green', )
progress = Progressbar(window, orient = HORIZONTAL,length = 300, mode = 'determinate', style = 'green.Horizontal.TProgressbar')
progress.grid()
progress.place(relx = 0.5, rely = 0.87, anchor = CENTER)

export = Button(window, text = 'Export To Excel', highlightbackground = bg, command = export, font = font)
export.grid()
export.place(relx = 0.5, rely = 0.95, anchor = CENTER)

window.mainloop()


credentials.json-drive-v3-[['https://www.googleapis.com/auth/drive']]
['https://www.googleapis.com/auth/drive']
token_drive_v3.pickle
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=656058805805-c4o8sv0e2fi0aj36cfeue8crormifji6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=q8a0m7WfXnTRgLD4DiAQ8XUOtH8ydY&access_type=offline
drive service created successfully
/Users/omarmarouf/Desktop/Data/Subject 1.xlsx
       Student Name                Student Email  Student ID
0       Dolly Cindi       DollyCindi27@gmail.com           1
1     Megan Carolee     MeganCarolee20@gmail.com           2
2        Lauri Cele        LauriCele27@gmail.com           3
3  Catriona Kynthia  CatrionaKynthia10@gmail.com           4
4     Shery Goldina    SherylGoldina27@gmail.com           5
Created folder Dolly Cindi with write access to DollyCindi27@gmai